In [1]:
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
import numpy as np
import os
from glob import glob
import os
from PIL import Image
import time
import numpy as np
import matplotlib.pyplot as plt
#import scipy
from keras.callbacks import ModelCheckpoint,LearningRateScheduler

Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [3]:
import math
def step_decay(epoch):
    
    
    initial_lrate=0.1
    drop=0.6
    epochs_drop = 3.0
    lrate= initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
   

lrate = LearningRateScheduler(step_decay)
callbacks_list = [ lrate]


In [4]:
latent_dim = 100

In [5]:
img_rows = 512
img_cols = 512
channels = 3
img_shape = (img_rows, img_cols, channels)

In [6]:
img_res=(512, 512)
dataset_name = 'PIX'
img_res = img_res
image_size = 512
img_rows = 512
img_cols = 512
channels = 3
img_shape = (img_rows, img_cols, channels)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)
disc_loss =[]
gen_loss = []
# Number of filters in the first layer of G and D
gf = 128
df = 128
optimizer = Adam(0.0001, 0.5)

In [7]:
def load_data( batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        if batch_size > 0:
            batch_images = np.random.choice(path, size=batch_size)
        else:
            batch_images = np.array([])
        #batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, img_res)
            img_B = scipy.misc.imresize(img_B, img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

In [8]:
def load_batch(batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        n_batches = int(len(path) / batch_size)

        for i in range(n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, img_res)
                img_B = scipy.misc.imresize(img_B, img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [9]:
def m_batch(batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        m_batches = int(len(path) / batch_size)
        return m_batches

In [10]:
def imread(path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)
    

In [11]:
def get_data(datadir):
    #datadir = args.data
    # assume each image is 512x256 split to left and right
    imgs = glob.glob(os.path.join(datadir, '*.jpg'))
    data_X = np.zeros((len(imgs),3,img_cols,img_rows))
    data_Y = np.zeros((len(imgs),3,img_cols,img_rows))	
    i = 0
    for file in imgs:
        img = cv2.imread(file,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (img_cols*2, img_rows)) 
        #print('{} {},{}'.format(i,np.shape(img)[0],np.shape(img)[1]))
        img = np.swapaxes(img,0,2)

        X, Y = split_input(img)

        data_X[i,:,:,:] = X
        data_Y[i,:,:,:] = Y
        i = i+1
    return data_X, data_Y

In [12]:
gf = 64
df = 64

optimizer = Adam(0.0002, 0.5)
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)


In [13]:
 def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

In [14]:
def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

In [15]:
 def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

In [16]:
def build_generator():
        """U-Net Generator"""

        

        # Image input
        d0 = Input(shape=img_shape)

        # Downsampling
        d1 = conv2d(d0, gf, bn=False)
        d2 = conv2d(d1, gf*2)
        d3 = conv2d(d2, gf*4)
        d4 = conv2d(d3, gf*8)
        d5 = conv2d(d4, gf*8)
        d6 = conv2d(d5, gf*8)
        d7 = conv2d(d6, gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, gf*8)
        u2 = deconv2d(u1, d5, gf*8)
        u3 = deconv2d(u2, d4, gf*8)
        u4 = deconv2d(u3, d3, gf*4)
        u5 = deconv2d(u4, d2, gf*2)
        u6 = deconv2d(u5, d1, gf)

        u7 = UpSampling2D(size=2)(u6)
        #noise = Input(shape=img_shape)
        output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

In [17]:
def build_generator1():

        d0 = Input(shape=img_shape)

        # Downsampling
        d1 = conv2d(d0, gf, normalize=False)
        d2 = conv2d(d1, gf*2)
        d3 = conv2d(d2, gf*4)
        d4 = conv2d(d3, gf*8)
        d5 = conv2d(d4, gf*8)
        d6 = conv2d(d5, gf*8)
        d7 = conv2d(d6, gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, gf*8)
        u2 = deconv2d(u1, d5, gf*8)
        u3 = deconv2d(u2, d4, gf*8)
        u4 = deconv2d(u3, d3, gf*4)
        u5 = deconv2d(u4, d2, gf*2)
        u6 = deconv2d(u5, d1, gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)


In [18]:
def build_discriminator():

        

        img = Input(shape=img_shape)

        d1 = d_layer(img, df, normalization=False)
        d2 = d_layer(d1, df*2)
        d3 = d_layer(d2, df*4)
        d4 = d_layer(d3, df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

In [ ]:
d_A = build_discriminator()
d_B = build_discriminator()
d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
d_B.compile(loss='mse',
            optimizer=optimizer,
metrics=['accuracy'])

In [ ]:
g_AB = build_generator()
g_BA = build_generator()
# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)

# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False
# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

In [ ]:
combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
optimizer=optimizer)


In [ ]:
def display_imageGA(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/genA',  exist_ok=True)
    pil_X.save("./images12/"+'genA/'+ t + ".png", 'PNG')

In [ ]:
def display_imageGB(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/genB',  exist_ok=True)
    pil_X.save("./images12/"+'genB/'+ t + ".png", 'PNG')

In [ ]:
def display_imageOA(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/oriA',  exist_ok=True)
    pil_X.save("./images12/"+'oriA/'+ t + ".png", 'PNG')

In [ ]:
def display_imageOB(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/oriB',  exist_ok=True)
    pil_X.save("./images12/"+'oriB/'+ t + ".png", 'PNG')

In [ ]:
def display_imageRB(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/RB',  exist_ok=True)
    pil_X.save("./images12/"+'RB/'+ t + ".png", 'PNG')

In [ ]:
def display_imageRA(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ((X*127.5+127.5).clip(0,255).astype('uint8'))
    int_X = int_X.reshape(-1,image_size,image_size, 3)
    int_X = int_X.reshape(rows, -1, image_size, image_size,3).swapaxes(1,2).reshape(rows*image_size,-1, 3)
    pil_X = Image.fromarray(int_X)
    t = str(round(time.time()))
    os.makedirs('./images12/RA',  exist_ok=True)
    pil_X.save("./images12/"+'RA/'+ t + ".png", 'PNG')

In [ ]:
 def sample_images( epoch, batch_i):
        os.makedirs('images/%s' % dataset_name, exist_ok=True)
        r, c = 3, 3
        batch_size=2
        imgs_A, imgs_B = load_data(batch_size=2, is_testing=True)

        # Translate images to the other domain
        fake_B = g_AB.predict(imgs_A)
        fake_A = g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = g_BA.predict(fake_B)
        reconstr_B = g_AB.predict(fake_A)
        display_imageGA(fake_B)
        display_imageOA(imgs_A)
        display_imageGB(fake_A)
        display_imageOB(imgs_B)
        display_imageRA(reconstr_A)
        display_imageRB(reconstr_B)
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[j,i].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images12/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
plt.close()

In [ ]:
def sample_images2(epoch, batch_i):
        os.makedirs('images/%s' % dataset_name, exist_ok=True)
        r, c = 3, 3
        batch_size=2
        imgs_A, imgs_B = load_data(batch_size=2, is_testing=True)
        fake_A = generator.predict(imgs_B)
        #print(imgs_A.shape)
        display_imageG(fake_A)
        display_imageO(imgs_A)
        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
        plt.close()

In [ ]:
def train( epochs, batch_size=128, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + disc_patch)
        fake = np.zeros((batch_size,) + disc_patch)

        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = g_AB.predict(imgs_A)
                fake_A = g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))
                d_A.save_weights('./DISCO/weights/d_a_weights2222.h5', True)
                d_B.save_weights('./DISCO/weights/d_b_weights2222.h5', True)
                g_AB.save_weights('./DISCO/weights/G_a_weights2222.h5', True)
                g_BA.save_weights('./DISCO/weights/G_b_weights2222.h5', True)
                combined.save_weights('./DISCO/weights/combined_weights2222.h5', True)

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    sample_images(epoch, batch_i)
                    #display_activation(activations, 16, 16, 3)



In [ ]:
train(epochs=700, batch_size=3, sample_interval=200)

/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `mode

[0] [0] time: 0:00:27.453353, [d_loss: 28.286407, g_loss: 62.168594]


/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/home/cvbl/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


[0] [1] time: 0:00:40.985033, [d_loss: 75.161606, g_loss: 91.167534]
[0] [2] time: 0:00:44.837357, [d_loss: 80.917084, g_loss: 7.415566]
[0] [3] time: 0:00:48.496716, [d_loss: 3.361418, g_loss: 9.549139]
[0] [4] time: 0:00:52.270981, [d_loss: 2.182079, g_loss: 6.944371]
[0] [5] time: 0:00:55.861799, [d_loss: 0.799877, g_loss: 4.638917]
[0] [6] time: 0:00:59.621080, [d_loss: 0.271640, g_loss: 3.931218]
[0] [7] time: 0:01:03.426982, [d_loss: 0.562211, g_loss: 4.219603]
[0] [8] time: 0:01:07.063661, [d_loss: 0.414094, g_loss: 3.701848]
[0] [9] time: 0:01:11.525025, [d_loss: 0.289803, g_loss: 3.499085]
[0] [10] time: 0:01:15.026316, [d_loss: 0.204118, g_loss: 3.362239]
[0] [11] time: 0:01:18.690816, [d_loss: 0.189948, g_loss: 3.384554]
[0] [12] time: 0:01:22.297892, [d_loss: 0.291650, g_loss: 3.753653]
[0] [13] time: 0:01:26.117904, [d_loss: 0.175308, g_loss: 3.382652]
[0] [14] time: 0:01:29.943931, [d_loss: 0.391886, g_loss: 3.507589]
[0] [15] time: 0:01:33.480429, [d_loss: 0.194397, g_lo